In [1]:
import networkx as nx
from tqdm import tqdm
from datetime import datetime
import numpy as np
import random


In [2]:
file = open('ca-AstroPh.txt','r')
lines = file.readlines()

In [3]:
def dic2list(myDic):
    graph = []
    for i in myDic.keys():
        graph.append(i)
    return graph

In [4]:
nodes = {}
unique_nodes = set()
neighbor_c = {}
for l in lines:
    if '#' in str(l):
        continue
    node = l.replace('\n','').split('\t')
    n1 = int(node[0])
    n2 = int(node[1])
    nodes[(n1,n2)] = 1
    unique_nodes.add(n1)
    unique_nodes.add(n2)
    if  neighbor_c.get(n1,0) == 0:
             neighbor_c[n1] = []
    neighbor_c[n1].append(n2)
nodes_set = dic2list(nodes)

In [5]:
def my_bfs(visited, queue, graph, node):
    visited.append(node)
    queue.append(node)
    n_list = []
    while queue:
        s = queue.pop(0)
        n_list.append(s)
        # print(graph[s])
        if graph.get(s,0) != 0:
            for neighbour in graph[s]:
                if neighbour not in visited:
                    visited.append(neighbour)
                    queue.append(neighbour)
    return n_list

In [6]:
def reward(visited, queue, graph, node,out):
    visited.append(node)
    queue.append(node)
    i = 0
    n_list = []
    while queue:
        s = queue.pop(0)
        n_list.append(s)
        if s == out:

            break
        i+=1
        if graph.get(s,0) != 0:
           for neighbour in graph.get(s,0):
                if neighbour not in visited:
                    visited.append(neighbour)
                    queue.append(neighbour)
    if out not in n_list or i == 0:
        i = 2
    return i

In [7]:
# graph = {5 : [3,7],3 : [2, 4],7 : [8],4 : [8],1 : [9],6:[]}
# print(reward([],[], graph, 1,9))

In [8]:
def generate_realization_graph(nodes,p):
    neighbor_probable = {}
    nodes_probable_unique = set()
    nodes_probable =  nodes.copy()
    for i in list(nodes_probable.keys()):
        if np.random.uniform(0,1,1)[0] > p:
            del nodes_probable[i]
        else:
            nodes_probable_unique.add(i[0])
            nodes_probable_unique.add(i[1])
            if  neighbor_probable.get(i[0],0) == 0:
                 neighbor_probable[i[0]] = []
            if  neighbor_probable.get(i[1],0) == 0:
                 neighbor_probable[i[1]] = []
            neighbor_probable[i[0]].append(i[1])

    return nodes_probable,nodes_probable_unique,neighbor_probable


In [9]:
nodes_p,nodes_p_unique,neighbor_p =  generate_realization_graph(nodes,0.015)
len(nodes_p)

5996

In [10]:
r_nodes_graph = list(nodes_p_unique)
random.shuffle(r_nodes_graph)
outbreak = r_nodes_graph[0:10]
outbreak

[107988, 50515, 79201, 29607, 40607, 77580, 37862, 72555, 79863, 1411]

In [11]:
my_bfs([],[],neighbor_p,28224)

[28224]

In [12]:
outbreack_neighbor = {}
# print(outbreak)
def outbreak_out(outbreak):
    for o in outbreak:
        # print(o)
        outbreack_neighbor[o] = my_bfs([],[],neighbor_p,o)
outbreak_out(outbreak)
outbreack_neighbor

{107988: [107988],
 50515: [50515, 8775, 114645, 60388],
 79201: [79201, 129998],
 29607: [29607],
 40607: [40607,
  6947,
  72975,
  111161,
  26988,
  5223,
  25452,
  34387,
  36907,
  40811,
  44502,
  67959,
  115607,
  120618,
  123634,
  37541,
  13068,
  52493,
  88216,
  99324,
  102035,
  124231,
  113210,
  11874,
  34066,
  131272,
  106794,
  79277,
  116488,
  80276,
  52192,
  97320,
  47968,
  118929,
  25037,
  80912,
  99718,
  66740,
  79860,
  116126,
  125302,
  83319,
  119615,
  65889,
  31290,
  31928,
  87214,
  40181,
  66514,
  72315,
  84327,
  89194,
  70898,
  17318,
  4786,
  52703,
  86892,
  129355,
  131028,
  123633,
  79187,
  41425,
  56723,
  57091,
  77415,
  69169,
  18642,
  55494,
  47314,
  130756,
  15973,
  97754,
  13889,
  27331,
  49944,
  95773,
  49242,
  22236,
  95318,
  96334,
  90427,
  74224,
  111211,
  114867,
  42220,
  117690,
  113825,
  113254,
  76438,
  1519,
  9226,
  34679,
  111639,
  13175,
  15575,
  111082,
  127193,


In [13]:
rewards = {}
def calculate_node_reward( ):
    for n in tqdm(nodes_p_unique):
        n_list = []
        for o in outbreak:
             n_list.append(-1 * reward([],[],neighbor_p,o,n))
        rewards[n] = np.sum(n_list)
        # print(n,n_list)
calculate_node_reward( )
# print(rewards)

100%|██████████| 6454/6454 [00:19<00:00, 333.29it/s]


In [14]:
# for k in sorted(rewards, key=lambda k: rewards[k], reverse=True)[0:10]:
#     print(k,rewards[k])

In [15]:
def calculate_nodes_reward(nodes_p_unique,neighbor_p,outbreak ):
    reward_new = {}
    for n in tqdm(nodes_p_unique):
        n_list = []
        for o in outbreak:
             n_list.append(-1 * reward([],[],neighbor_p,o,n))
        reward_new[n] = np.sum(n_list)
    return reward_new

In [16]:
def hill_climing():
    start = datetime.now()
    sort =  sorted(rewards, key=lambda k: rewards[k], reverse=True)
    S = []
    cost=[]
    neighbor_p_c = neighbor_p.copy()
    S.append(sorted(sort)[0])
    cost.append(len(neighbor_p_c[sort[0]]) * 0.8)
    unique_nodes_h = nodes_p_unique.copy()
    outbreak_h = outbreak.copy()
    outbreak_h.pop(0)
    neigh = set()
    for s in S:
        unique_nodes_h.remove(s)
        del neighbor_p_c[s]
    while np.sum(cost) < 5 :
        temp_dic = {}
        temp_dic.update(calculate_nodes_reward(unique_nodes_h,neighbor_p_c,outbreak_h))
        n = sorted(temp_dic, key=lambda k: temp_dic[k], reverse=True)[0]
        S.append(n)
        cost.append(len(neighbor_p[n]) * 0.8)
        unique_nodes_h.remove(n)
        del neighbor_p_c[n]
        if len(outbreak_h) == 0:
            break
        outbreak_h.pop(0)
    print("hill climbing time : ", datetime.now()-start)
    return S


for k in hill_climing():
    print(k,rewards[k])

100%|██████████| 6444/6444 [00:00<00:00, 266436.27it/s]

hill climbing time 0:01:17.636883
13 -20
100450 -19
6947 -19
40000 -19
81761 -19
61429 -403
65540 -20
81925 -20
49159 -20
98315 -20
49163 -20


### Lazy hill climbing

In [17]:
mg_sorted = sorted(rewards, key=lambda k: rewards[k], reverse=True)
mg_sorted[0:10]

[100450, 6947, 40000, 81761, 129998, 65540, 98315, 13, 15, 98325]

In [18]:
def calculate_node_reward(node,neighbor_p,outbreak ):
    reward_new = {}

    n_list = []
    for o in outbreak:
         n_list.append(-1 * reward([],[],neighbor_p,o,node))
    reward_new[node] = np.sum(n_list)
    return reward_new

In [19]:
def lazy_hill_climbing():
    start = datetime.now()
    cost =  []
    S= {}
    mg_lh_sorted = mg_sorted.copy()
    # S.append(mg_lh_sorted[0])
    unique_nodes_h = nodes_p_unique.copy()
    outbreak_h = outbreak.copy()
    neighbor_p_c = neighbor_p.copy()
    rewards_lhc = rewards.copy()
    S[mg_lh_sorted[0]] = rewards_lhc[mg_lh_sorted[0]]
    outbreak_h.pop(0)
    cost.append(len(neighbor_p_c[mg_lh_sorted[0]]) * 0.8)
    for i , n in enumerate(mg_lh_sorted):
        if len (S) > 9:
            break
        left = n
        right = mg_sorted[i+1]
        l = rewards_lhc[left]
        r = rewards_lhc[right]
        if np.sum(cost) < 10 :
            if i != 0:
                S[n] = rewards_lhc[n]
            rewards_lhc[n] = calculate_node_reward(mg_lh_sorted[i+1],neighbor_p_c,outbreak_h)
            unique_nodes_h.remove(n)
            del neighbor_p_c[n]
            cost.append(len(neighbor_p[n]) * 0.8)
            if len(outbreak_h) == 0:
                break
        else:
            neighbor_p_c = sorted(rewards_lhc, key=lambda k: rewards_lhc[k], reverse=True)
    print(S)
    print("lazy hill climbing time : ", datetime.now()-start)
    return S
lhc_simple = lazy_hill_climbing()

{100450: -19, 6947: -19, 40000: -19, 81761: -19, 129998: -19, 65540: -20, 98315: -20, 13: -20, 15: -20, 98325: -20}
lazy hill climbing time :  0:00:00.031425


### Celf with benefit cost

In [20]:
def calculate_benefit_cost(node,neighbor_p,outbreak ):
    reward_new = {}
    # cost = {}
    n_list = []
    for o in outbreak:
        if node in outbreack_neighbor[o]:
                n_list.append(-1 * reward([],[],neighbor_p,o,node))
        else:
                n_list.append(0)

    reward_new[node] = (np.sum(n_list) * 0.25 * -1)/(0.8* len(neighbor_p[node]))
    # cost[node] = (0.8* len(neighbor_p[node]))
    return reward_new

In [21]:
rewards={}
def calculate_node_reward( ):

    for n in tqdm(nodes_p_unique):
        n_list = []
        for o in outbreak:
             n_list.append(-1 * reward([],[],neighbor_p,o,n))
        rewards[n] = (np.sum(n_list) * 0.25 * -1)/0.8* len(neighbor_p[n])
        # print(n, rewards[n])
    return rewards

x = calculate_node_reward()
mg_celf_bc_sorted = sorted(x, key=lambda k: x[k], reverse=True)

# mg_celf_bc_sorted

100%|██████████| 6454/6454 [00:19<00:00, 333.57it/s]


In [22]:
def celf_bc():
    start = datetime.now()
    S = {}
    cost = []
    rewards_lhc = x.copy()
    mg_lh_sorted = mg_celf_bc_sorted.copy()
    S[mg_lh_sorted[0]] = rewards_lhc[mg_lh_sorted[0]]
    unique_nodes_h = nodes_p_unique.copy()
    outbreak_h = outbreak.copy()
    neighbor_p_c = neighbor_p.copy()
    cost.append(len(neighbor_p_c[mg_lh_sorted[0]]) * 0.8)
    outbreak_h.pop(0)
    for i , n in enumerate(mg_lh_sorted):
        if len (S) > 9:
            break
        left = n
        right = mg_lh_sorted[i+1]
        l = rewards_lhc[left]
        r = rewards_lhc[right]
        # print(np.sum(cost))
        # if True :
        if l > r :
            if i != 0:
                # S.append(n)
                # print(rewards_lhc[n])
                S[n] = rewards_lhc[n]
            rewards_lhc[n] = calculate_benefit_cost(mg_lh_sorted[i+1],neighbor_p_c,outbreak_h)
            cost.append(len(neighbor_p_c[n]) * 0.8)
            unique_nodes_h.remove(n)
            del neighbor_p_c[n]
            if len(outbreak_h) == 0 or np.sum(cost) > 10 :
                break

        else:
            neighbor_p_c = sorted(rewards_lhc, key=lambda k: rewards_lhc[k], reverse=True)
    print(S)
    print("benefit cost time: ",datetime.now()-start)
    return S


lhc_benefit = celf_bc()


{128081: 1803.75, 53213: 1774.6875}
benefit cost time:  0:00:00.004840


In [23]:
# colors = {
#     "blue" : "5",
#     "red" : "6",
#     "yellow" : "8",
# }
# first_key = list(colors)[0]
# first_val = list(colors.values())[0]
# print(first_key,first_val)
lhc_benefit

{128081: 1803.75, 53213: 1774.6875}

In [24]:
for i in range(len(lhc_simple)):
    if len(lhc_benefit) > i  :
        if list(lhc_simple.values())[i] > list(lhc_benefit.values())[i]:
            print(list(lhc_simple)[i])
        else:
            print(list(lhc_benefit)[i])
    else:
        print(list(lhc_simple)[i])


128081
53213
40000
81761
129998
65540
98315
13
15
98325
